<a href="https://colab.research.google.com/github/imk1raa/imk1raa/blob/main/kira_afremov_ex2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## SOURCE

In [35]:
%load_ext tensorboard
import datetime, os
import tensorflow as tf
import tensorflow_datasets as tfds


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


DOWNLOADING & PREPARING DATASET

In [36]:
(ds_train, ds_test), ds_info = tfds.load('mnist',
          split=['train', 'test'],
          shuffle_files=True,
          as_supervised=True,
          with_info=True,)

num_of_classes = ds_info.features['label'].num_classes
image_shape = ds_info.features['image'].shape
train_size = ds_info.splits['train'].num_examples
test_size = ds_info.splits['test'].num_examples


print(num_of_classes)
print(image_shape)
print(train_size)
print(test_size)


10
(28, 28, 1)
60000
10000


DATA PREPROCESING

In [37]:
def normalize_img(image, label):
    "Normalizes images: `uint8` -> `float32`"
    return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(32)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)


ds_test = ds_test.map(
     normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

In [55]:
layers = [
        tf.keras.layers.Flatten(input_shape=image_shape),

        tf.keras.layers.Dense(87, kernel_regularizer=tf.keras.regularizers.l2(0.0001)),
        tf.keras.layers.Activation('elu'),
        tf.keras.layers.Dropout(0.1),

        tf.keras.layers.Dense(30, kernel_regularizer=tf.keras.regularizers.l2(0.001)),
        tf.keras.layers.Activation('elu'),
        tf.keras.layers.Dropout(0.2),

        tf.keras.layers.Dense(64, kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        tf.keras.layers.Activation('elu'),
        tf.keras.layers.Dropout(0.3),

        tf.keras.layers.Dense(37, kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        tf.keras.layers.Activation('elu'),
        tf.keras.layers.Dropout(0.3),

        tf.keras.layers.Dense(num_of_classes),
        tf.keras.layers.Softmax()
        ]

SUMMERY + COMPILE

In [56]:
model = tf.keras.models.Sequential(layers)

model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.summary()

Model: "sequential_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten_14 (Flatten)                 │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_56 (Dense)                     │ (None, 87)                  │          68,295 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_42 (Activation)           │ (None, 87)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_20 (Dropout)                 │ (None, 87)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_57 (Dense)                     │ (None, 30)                  │           2,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_43 (Activation)           │ (None, 30)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_21 (Dropout)                 │ (None, 30)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_58 (Dense)                     │ (None, 64)                  │           1,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_44 (Activation)           │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_22 (Dropout)                 │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_59 (Dense)                     │ (None, 37)                  │           2,405 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_45 (Activation)           │ (None, 37)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_23 (Dropout)                 │ (None, 37)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_60 (Dense)                     │ (None, 10)                  │             380 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ softmax_13 (Softmax)                 │ (None, 10)                  │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 75,704 (295.72 KB)

 Trainable params: 75,704 (295.72 KB)

 Non-trainable params: 0 (0.00 B)

EPOCH'S

In [57]:
model.fit(ds_train,
          epochs=50,
          validation_data=ds_test)

Epoch 1/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - loss: 1.4058 - sparse_categorical_accuracy: 0.7601 - val_loss: 0.4388 - val_sparse_categorical_accuracy: 0.9255
Epoch 2/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - loss: 0.5026 - sparse_categorical_accuracy: 0.9142 - val_loss: 0.3474 - val_sparse_categorical_accuracy: 0.9464
Epoch 3/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - loss: 0.4273 - sparse_categorical_accuracy: 0.9296 - val_loss: 0.3072 - val_sparse_categorical_accuracy: 0.9554
Epoch 4/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 0.3757 - sparse_categorical_accuracy: 0.9422 - val_loss: 0.2936 - val_sparse_categorical_accuracy: 0.9562
Epoch 5/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - loss: 0.3589 - sparse_categorical_accuracy: 0.9432 - val_loss: 0.2745 - val_sparse_categorical_accuracy: 0.9598
Epoch 6/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - loss: 0.3370 - sparse_categorical_accuracy: 0.9486 - val_loss: 0.2634 - val_sparse_categorical_accurac

KeyboardInterrupt: 